In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM
from keras.layers import Dropout
from os.path import isfile, join

2021-11-30 11:44:44.022623: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-11-30 11:44:44.022671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
dft = pd.read_csv('dataGendb4.csv')
dat = []
files = dft["file"]
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label

In [14]:
from keras.layers import Flatten, Conv1D, Input, MaxPooling1D, Conv2D, MaxPooling2D, BatchNormalization, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

In [ ]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(x_train),np.array(y_train),stratify=y_train,test_size = 0.3,random_state=777,shuffle=True)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit(x_tr, y_tr ,epochs=100, callbacks=[es,mc], batch_size=32, validation_data=(x_val,y_val))

In [15]:
from tensorflow.keras.models import Model, load_model
from keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import layers as L
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
import pickle
import os
from os import listdir
import pywt
from pywt import dwt, wavedec
from os.path import isfile, join
import torch
import tensorflow as tf
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
sample_rate = 44100
# from kapre.time_frequency import Melspectrogram, Spectrogram
# from kapre.utils import Normalization2D
from kapre.composed import get_melspectrogram_layer
def AttRNNSpeechModel(rnn_func=L.LSTM):
    # simple LSTM
#     sr = samplingrate
#     iLen = inputLength
    inputs = Input(shape=(7, 20, 216))
#     top_db = 80
    
#     spec = transforms.MelSpectrogram(44100, n_fft=1024, hop_length=hop_size, n_mels=32)(torch.tensor(tf.make_ndarray(tf.make_tensor_proto(inputs))))
#     spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
#     print(spec.size())
#     x = MelSpec()(inputs)

#     x = get_melspectrogram_layer(n_fft=2048, win_length=2048, hop_length=256, sample_rate=44100, n_mels=13, input_data_format='channels_first')(inputs)
#     print(x)
    x = L.Conv2D(256, (5, 1), activation='relu', padding='same')(inputs)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(128, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(64, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(32, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(16, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)
    x = L.Conv2D(1, (5, 1), activation='relu', padding='same')(x)
    x = L.BatchNormalization()(x)

    # x = Reshape((125, 80)) (x)
    # keras.backend.squeeze(x, axis)
    x = L.Lambda(lambda q: K.squeeze(q, -1), name='squeeze_last_dim')(x)

    x = L.Bidirectional(rnn_func(128, return_sequences=True)
                        )(x)  # [b_s, seq_len, vec_dim]
    x = L.Bidirectional(rnn_func(64, return_sequences=True)
                        )(x)  # [b_s, seq_len, vec_dim]

    xFirst = L.Lambda(lambda q: q[:, -1])(x)  # [b_s, vec_dim]
    query = L.Dense(128)(xFirst)

    # dot product attention
    attScores = L.Dot(axes=[1, 2])([query, x])
    attScores = L.Softmax(name='attSoftmax')(attScores)  # [b_s, seq_len]

    # rescale sequence
    attVector = L.Dot(axes=[1, 1])([attScores, x])  # [b_s, vec_dim]

    x = L.Dense(64, activation='relu')(attVector)
    x = L.Dense(32, activation='relu')(attVector)
    x = L.Dense(16)(x)

    output = L.Dense(84, activation='softmax', name='output')(x)

    model = Model(inputs=[inputs], outputs=[output])

    return model

model = AttRNNSpeechModel()#, rnn_func=L.LSTM)

model.compile(optimizer='adam', loss=['categorical_crossentropy'], metrics=['accuracy'])
model.summary()

2021-11-30 11:44:54.567754: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-11-30 11:44:54.568705: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-11-30 11:44:54.569307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64:/usr/local/cuda/lib64
2021-11-30 11:44:54.569754: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libc

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 20, 216)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 7, 20, 256)   276736      ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 7, 20, 256)  1024        ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

In [16]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size, validation,
                 shuffle=True):

        self.df = df.copy()
        if validation:
            self.df = self.df[25000:]
        else:
            self.df = self.df[:25000]
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
    
        objects = []
        with (open(path, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        return objects[0]["audio"]

    
    def __get_output(self, tag):
        return lab[tag]
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        files = batches["file"]
        tag = batches["tag"]

        X_batch = np.asarray([self.__get_input(x) for x in files])

        y_batch = np.asarray([self.__get_output(y) for y in tag])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
from sklearn.model_selection import train_test_split
# x_tr, x_val, y_tr, y_val = train_test_split(np.array(x_train),np.array(y_train),stratify=y_train,test_size = 0.3,random_state=777,shuffle=True)
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.00001) 
from sklearn.utils import shuffle
dft = shuffle(dft)
training_generator = CustomDataGen(dft, 32, False)
validation_generator = CustomDataGen(dft, 32, True)
mc = ModelCheckpoint('best_model.hdf5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=mc, use_multiprocessing=True, workers=6)

/tmp/ipykernel_347/4152564705.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=training_generator, validation_data = validation_generator, epochs=50, callbacks=mc, use_multiprocessing=True, workers=6)


Epoch 1/50


2021-11-30 11:45:06.090759: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 80 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 28835840 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


781/781 [==============================] - ETA: 0s - loss: 2.5167 - accuracy: 0.3439

2021-11-30 11:47:14.858754: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "101" frequency: 2100 num_cores: 80 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 1048576 l3_cache_size: 28835840 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }



Epoch 00001: val_accuracy improved from -inf to 0.57043, saving model to best_model.hdf5
781/781 [==============================] - 149s 183ms/step - loss: 2.5167 - accuracy: 0.3439 - val_loss: 1.5040 - val_accuracy: 0.5704
Epoch 2/50
781/781 [==============================] - ETA: 0s - loss: 0.9491 - accuracy: 0.7107
Epoch 00002: val_accuracy improved from 0.57043 to 0.78109, saving model to best_model.hdf5
781/781 [==============================] - 155s 196ms/step - loss: 0.9491 - accuracy: 0.7107 - val_loss: 0.7039 - val_accuracy: 0.7811
Epoch 3/50
781/781 [==============================] - ETA: 0s - loss: 0.5948 - accuracy: 0.8044
Epoch 00003: val_accuracy improved from 0.78109 to 0.79145, saving model to best_model.hdf5
781/781 [==============================] - 158s 201ms/step - loss: 0.5948 - accuracy: 0.8044 - val_loss: 0.6652 - val_accuracy: 0.7915
Epoch 4/50
781/781 [==============================] - ETA: 0s - loss: 0.4339 - accuracy: 0.8526
Epoch 00004: val_accuracy improve

In [ ]:
import torch
import torchaudio
from torchaudio import transforms
def spectro_gram(aud, n_mels=32, n_fft=2048, hop_len=None):
    sig,sr = aud
    top_db = 80
    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

In [ ]:
import pandas as pd
import os
import librosa
import scipy
filePath = "./saraga1.5_hindustani"
csv_path = os.path.join(filePath, "file_paths.csv")
df = pd.read_csv(csv_path)

files = []
pPath = "./saraga1.5_hindustani/"
paths = []
for i in df["filepath"]:
    mod_path = i.split("/")
    paths.append(pPath+mod_path[0]+"/"+mod_path[1])
for fold in paths:
    for file in os.listdir(fold):
        if file.endswith(".wav"):
            full_file_path = fold+"/"+file
            files.append(full_file_path)

In [ ]:
import pickle
import os
from os import listdir
import pywt
from pywt import dwt, wavedec
from os.path import isfile, join
import torch
less = 0
good = 0
window_size = 2048
hop_size =256
allLabels = []
# toSpec = np.zeros((7, 1323000), dtype=np.float32)
# samples, sample_rate = torchaudio.load('./saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/97.wav')
# # print(samples.size())
# sam = wavedec(samples, 'db8', level=5)
# print(np.shape(sam[0]))
# toSpec[6] = samples.cpu().detach().numpy()
# toSpec[5,::2] = sam[5][0][3:661503]
# toSpec[4,::4] = sam[4][0][10:330760]
# toSpec[3,::8] = sam[3][0][7:165382]
# toSpec[2,::16] = sam[2][0][8:82696]
# toSpec[1,::32] = sam[1][0][7:41351]
# toSpec[0,::32] = sam[0][0][7:41351]
# print(torch.tensor(toSpec))

# mfcc = spectro_gram((torch.tensor(toSpec), sample_rate), hop_len = hop_size)
# print(torch.nn.functional.normalize(mfcc))
with open("ragas10.pickle", 'wb') as handle:
    for j in files:
        label = j.split("/")[-2]
        samples, sample_rate = torchaudio.load(j)
        if len(samples[0]) == 1323000:
            toSpec = np.zeros((7, 1323000), dtype=np.float32)
            sam = wavedec(samples, 'db8', level=5)
            toSpec[6] = samples.cpu().detach().numpy()
            toSpec[5,::2] = sam[5][0][3:661503]
            toSpec[4,::4] = sam[4][0][10:330760]
            toSpec[3,::8] = sam[3][0][7:165382]
            toSpec[2,::16] = sam[2][0][8:82696]
            toSpec[1,::32] = sam[1][0][7:41351]
            toSpec[0,::32] = sam[1][0][7:41351]
            mfcc = spectro_gram((torch.tensor(toSpec), sample_rate), hop_len = hop_size)
            pickle.dump({label : torch.nn.functional.normalize(mfcc).cpu().detach().numpy()}, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
import pandas as pd
import os
import librosa
import scipy
filePath = "./saraga1.5_hindustani"
csv_path = os.path.join(filePath, "file_paths.csv")
df = pd.read_csv(csv_path)
raga_filenames = []

for i in range(108):
    raga_filenames.append(os.path.join(filePath, df["filepath"][i])+".mp3.mp3")
    
files = []
pPath = "./saraga1.5_hindustani/"
paths = []
for i in df["filepath"]:
    mod_path = i.split("/")
    print(pPath+mod_path[0]+"/"+mod_path[1])
    paths.append(pPath+mod_path[0]+"/"+mod_path[1])
for fold in paths:
    tag = fold.split('/')[-1]
    for file in os.listdir(fold):
        if file.endswith("db4.pickle"):
            full_file_path = fold+"/"+file
            files.append(full_file_path)

./saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree
./saraga1.5_hindustani/Raag Lagan Gandhar & Nirgun Bhajan by Kumar Gandharva/Nirgun Bhajan
./saraga1.5_hindustani/Raag Lagan Gandhar & Nirgun Bhajan by Kumar Gandharva/Raag Lagan Gandhar
./saraga1.5_hindustani/Raag Kalyan, Bhimpalasi & Dhani by Satyasheel Deshpande/Raag Dhani
./saraga1.5_hindustani/Raag Kalyan, Bhimpalasi & Dhani by Satyasheel Deshpande/Raag Bhimpalasi
./saraga1.5_hindustani/Raag Kalyan, Bhimpalasi & Dhani by Satyasheel Deshpande/Raag Kalyan
./saraga1.5_hindustani/Raag Bageshree & Bhairavi Bhajan by Satyasheel Deshpande/Bhairavi Bhajan
./saraga1.5_hindustani/Raag Bageshree & Bhairavi Bhajan by Satyasheel Deshpande/Raag Bageshree
./saraga1.5_hindustani/Raag Kedar & Jog by Ajoy Chakrabarty/Raag Kedar
./saraga1.5_hindustani/Raag Kedar & Jog by Ajoy Chakrabarty/Raag Jog
./saraga1.5_hindustani/Hindustani Alaps by Kaustuv Kanti Ganguli/Raag Lalit
./saraga1.5_hindustani/Hindustani Alaps by Kaustuv Kanti Ga

In [4]:
files

['./saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/8987db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/7467db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/1121db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/3154db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/4465db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/6080db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/836db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/9823db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/6308db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/608db4.pickle',
 './saraga1.5_hindustani/Raag Shree by Deborshee Bhattacharya/Raag Shree/9595db4.pickle',
 './saraga1.

In [5]:
# data = [f.split("/")[-2] for f in files if ]
dat = []
for i in files:
    if i.split("/")[-2] not in dat:
        dat.append(i.split("/")[-2])

# header = ["file", "tag"]
# import csv

# with open('dataGen.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)

#     # write the data
#     writer.writerows(data)

In [6]:
len(dat)

84

In [ ]:
# x_train = np.zeros((10150,7,661500))
# y_train = np.zeros((10150, 78))
import pickle
unique = 0
d = []
file = "./saraga1.5_hindustani/Raag Bihag & Bhairavi Dadra by Omkar Dadarkar/Bhairavi Dadra/5.pickle"

objects = []
tag = file.split("/")[-2]
d.append(tag)
with (open(file, "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
            
#     x_train[i] = objects[0]["audio"]
#     y_train[i] = lab[tag]

In [7]:
import numpy as np
# len(objects[0]['audio'][0])

In [ ]:
np.shape(objects[0]['audio'])

In [8]:
lab = {}

for i in range(84):
    tag = dat[i]
    y_label = np.zeros((84))
    y_label[i] = 1
    lab[tag] = y_label

In [ ]:
lab

In [9]:
data = [[file, file.split("/")[-2]] for file in files]
header = ["file", "tag"]
import csv
with open('dataGendb4.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

In [10]:
dft = pd.read_csv('dataGendb4.csv')

In [11]:
len(dft)

31201

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df,
                 batch_size, validation,
                 shuffle=True):

        self.df = df.copy()
        if validation:
            self.df = self.df[30000:]
        else:
            self.df = self.df[:30000]
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path):
    
        objects = []
        with (open(path, "rb")) as openfile:
            while True:
                try:
                    objects.append(pickle.load(openfile))
                except EOFError:
                    break
        return objects[0]["audio"]

    
    def __get_output(self, tag):
        return lab[tag]
    
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        files = batches["file"]
        print(files)
        tag = batches["tag"]

        X_batch = np.asarray([self.__get_input(x) for x in files])

        y_batch = np.asarray([self.__get_output(y) for y in tag])

        return X_batch, y_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
t = model.predict(np.reshape(objects[0]['audio'], (1, 7,20,216)))

In [ ]:
t

In [ ]:
lab

In [ ]:
np.where(t == np.max(t))

In [ ]:
objects = []
import pickle
with (open("test.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
objects = []
import pickle
with (open("google10wavelet.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break
labels = ["bed", "bird", "cat", "dog", "down", "eight", "five", "follow", "forward", "four"]
print(len(labels))
lab = {}
for i in range(len(labels)):
    lab[labels[i]] = np.zeros((10))
    lab[labels[i]][i] = 1

x_train = []
y_train = []
for i in range(len(objects)):
    tag = [*objects[i]][0].split('/')[0]
    if tag not in ("go", "happy", "house", "learn", "left", "marvin", "nine", "no", "off", "on", "one", "right", "seven", "sheila", "six", "stop", "three", "two", "up", "visual", "wow", "yes", "zero"):
        x_train.append(objects[i][[*objects[i]][0]])
        y_train.append(lab[[*objects[i]][0].split('/')[0]])